# Datafaucet

Datafaucet is a productivity framework for ETL, ML application. Simplifying some of the common activities which are typical in Data pipeline such as project scaffolding, data ingesting, start schema generation, forecasting etc.

In [33]:
from kafka import KafkaConsumer, TopicPartition
from json import loads, dumps
from time import sleep

In [34]:
consumer = KafkaConsumer(
    'dfc',
     bootstrap_servers=['kafka:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=False,
     group_id=None,
     value_deserializer=lambda x: loads(x.decode('utf-8')))

In [35]:
p = consumer.partitions_for_topic('dfc')
partitions = [TopicPartition('dfc', x) for x in list(p)]
partitions

[TopicPartition(topic='dfc', partition=0)]

In [36]:
consumer.beginning_offsets(partitions)

{TopicPartition(topic='dfc', partition=0): 0}

In [59]:
while not list(consumer.end_offsets(partitions).values())[0]:
    print(consumer.end_offsets(partitions))
    sleep(1)

consumer.end_offsets(partitions)

{TopicPartition(topic='dfc', partition=0): 16}

In [60]:
consumer.seek_to_beginning(partitions[0])

In [61]:
message = consumer.poll(timeout_ms=1000, max_records=1)
cnt = len(message.values())
if cnt:
    record = list(message.values())[0][0]
    print(f'[consumer]: topic={record.topic}, partition={record.partition}, offset={record.offset}, timestamp={record.timestamp}')
    print(f'[datafaucet log data]:')
    print(dumps(record.value, indent=2))
else:
    print('No data in the queue')

[consumer]: topic=dfc, partition=0, offset=0, timestamp=1576410421510
[datafaucet log data]:
{
  "@timestamp": "2019-12-15T11:47:01.162778",
  "severity": "INFO",
  "sid": "0x9b07dc541f3011ea",
  "repohash": 0,
  "reponame": "",
  "username": "jovyan",
  "filepath": "../../../opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py",
  "funcname": "notebook:cell",
  "message": "info",
  "data": null
}


In [75]:
consumer.beginning_offsets(partitions)
consumer.seek(partitions[0], 7)
consumer.position(partitions[0])

7

In [77]:
message = consumer.poll(timeout_ms=1000, max_records=1)

cnt = len(message.values())
d = {'severity':None, 'message':None, 'data':None}

if cnt:
    record = list(message.values())[0][0]
    print(f'[consumer]: topic={record.topic}, partition={record.partition}, offset={record.offset}, timestamp={record.timestamp}')
    print(f'[datafaucet log data]:')
    print(dumps(record.value, indent=2))
    d = record.value
else:
    print('No data in the queue')

[consumer]: topic=dfc, partition=0, offset=8, timestamp=1576410906639
[datafaucet log data]:
{
  "@timestamp": "2019-12-15T11:55:06.634726",
  "severity": "INFO",
  "sid": "0xbc817b3c1f3111ea",
  "repohash": 0,
  "reponame": "",
  "username": "jovyan",
  "filepath": "logging.ipynb",
  "funcname": "notebook:cell",
  "message": "info",
  "data": null
}


In [74]:
assert d['severity'] == 'WARNING'
assert d['message'] == 'custom data + message'
assert d['data'] == {'test_value': 42}

AssertionError: 